<a href="https://colab.research.google.com/github/Rekt77/kisa_insuretech/blob/master/pytorch_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
# 한글 형태소 분석기 다운로드
!pip install konlpy

In [13]:
#형태소 단위로 문장 분해
from konlpy.tag import Kkma
kor_nlp = Kkma()
kor_nlp.morphs("운전을 배우고 있는 중")

['운전', '을', '배우', '고', '있', '는', '중']

In [0]:
import torch.nn as nn
import torch
from torch import optim
from konlpy.tag import Kkma
from torch.autograd import Variable
kor_nlp = Kkma()

In [0]:
# 데이터 셋 저장
# 나만의 데이터셋 만들기: 웹 사이트에서 글 크롤링
# 운전자 커뮤니티 -> 운전자
# 건강,헬스 커뮤니티 -> 건강
# 등등..

train_data = [["운전을 배우고 있는 중","운전자"],
              ["몸에 좋은거 챙길 나이야","건강"],
              ["어제 다리가 부러졌어","상해"],
              ["어제 사고가 났어","운전자"],
              ["차 사줘","운전자"],
              ["몸이 아픈거 같아","건강"],
              ["다리 아파","상해"],
              ["뼈가 부러졌어","상해"],
              ["손목이 안 좋아","상해"],
              ["병원 아는데 있어?","건강"],
              ["요즘 병원에 자주가게 되네","건강"],
              ["건강검진 받고 몸에 좋은거 챙겨","건강"],
              ["어제 운전한거 블랙박스 확인해봐","운전자"],
              ["손목 아프면 병원에 가야지","상해"]]

test_data = [["운전이 하고싶어","운전자"],
             ["건강식 챙겨먹고 있어","건강"],
             ["뼈가 부러져서 금이 갔어","상해"],
             ["사고나면 어떻게 해야돼?","운전자"]]

In [16]:
# train_X = 문장 >> ('hi my name is', 'hi your name is')
# train_Y = 분류 >> ('운전자','상해')
train_X,train_y = list(zip(*train_data))

# 토큰화
# 꼬꼬마 형태소 분석기를 이용해서 문장을 분석
# 형태소 단위로 분해하여 train_x에 저장
train_X = [kor_nlp.morphs(x) for x in train_X]

# 단어별 인덱스를 만들것임. 모르는 단어는 unknown
word2index={'unknown' : 0}

# 문장을 추출
for x in train_X:
  # 형태소단위로 분해된 토큰을 추출
    for token in x:
      #word2index에 추가되지 않은 단어이면 단어를 추가
        if word2index.get(token)==None:
          #추가 할때마다 word2index의 길이를 인덱스로 반영
            word2index[token]=len(word2index)

# 클래스를 인덱스화
# 클래스가 3개 이므로 길이 3의 딕셔너리를 생성
class2index = {'운전자' : 0, '상해' : 1, '건강' : 1}
print(word2index)
print(class2index)

{'unknown': 0, '운전': 1, '을': 2, '배우': 3, '고': 4, '있': 5, '는': 6, '중': 7, '몸': 8, '에': 9, '좋': 10, '은': 11, '거': 12, '챙기': 13, 'ㄹ': 14, '나이': 15, '야': 16, '어제': 17, '다리': 18, '가': 19, '부러지': 20, '었': 21, '어': 22, '사고': 23, '나': 24, '차': 25, '사': 26, '아': 27, '주': 28, '이': 29, '아프': 30, 'ㄴ': 31, '같': 32, '뼈': 33, '손목': 34, '안': 35, '좋아': 36, '병원': 37, '알': 38, '데': 39, '?': 40, '요즘': 41, '자주': 42, '가게': 43, '되': 44, '네': 45, '건강': 46, '검진': 47, '받': 48, '하': 49, '블랙': 50, '박스': 51, '확인': 52, '해보': 53, '면': 54, '야지': 55}
{'운전자': 0, '상해': 1, '건강': 1}


In [0]:
# Bag of Words 만들기
def make_BoW(seq,word2index):

  #보유한 단어 수 크기의 0으로 초기화된 텐서 구현
    tensor = torch.zeros(len(word2index))

    #seq는 형태소 문장
    #w는 나누어진 형태소
    for w in seq:
      # word2index로 부터 w의 인덱스를 구함
        index = word2index.get(w)

        # 인덱스가 있을 경우
        if index!=None:
          # 해당 인덱스의 값을 1로 만들어줌
            tensor[index]+=1.

        # 인덱스가 없는경우
        else:
          # unknown의 값이 인덱스가 되고
          # 해당 인덱스의 값을 1로 만들어줌
            index = word2index['unknown']
            tensor[index]+=1.
    
    return tensor

# ['운전', '을', '배우', '고', '있', '는', '중']의 인덱스가 0,1,2,3,4,5,6 이라면
# 0~6번째 요소가 1로 채워진 텐서가 만들어짐

train_X = torch.cat([Variable(make_BoW(x,word2index)).view(1,-1) for x in train_X])
train_y = torch.cat([Variable(torch.LongTensor([class2index[y]])) for y in train_y])

In [0]:
# BoW classifier 만들기
class BoWClassifier(nn.Module):
    def __init__(self,vocab_size,output_size):
        super(BoWClassifier,self).__init__()
        
        # 뉴런간 선형결합
        # vocab을 아웃풋에 연결하여 연산
        self.linear = nn.Linear(vocab_size,output_size)
    
    def forward(self,inputs):
        return self.linear(inputs)

In [22]:
# 학습
epochs = 1000
LR = 0.1

# 뉴런이 
# 인풋 크기는 word2index의 길이
# 아웃풋은 3(클래스가 3개이기 때문)
model = BoWClassifier(len(word2index),3)

# cost함수 선언
loss_function = nn.CrossEntropyLoss()

# 경사하강법으로 학습
optimizer = optim.SGD(model.parameters(),lr=LR)

for epoch in range(epochs):
    model.zero_grad()
    hypothesis = model(train_X)
    loss = loss_function(hypothesis,train_y)
    if epoch % 10 == 0:
        print(loss.item())
    loss.backward()
    optimizer.step()

1.1282328367233276
0.6489270925521851
0.4664818346500397
0.36661818623542786
0.30189600586891174
0.25619441270828247
0.22213223576545715
0.19575969874858856
0.17474837601184845
0.15762899816036224
0.14342544972896576
0.13146254420280457
0.1212579756975174
0.11245784908533096
0.10479658842086792
0.09807093441486359
0.09212292730808258
0.0868278294801712
0.08208588510751724
0.07781654596328735
0.07395388931035995
0.07044364511966705
0.06724061071872711
0.06430692225694656
0.06161060929298401
0.059124529361724854
0.05682545527815819
0.05469341203570366
0.05271118879318237
0.05086377635598183
0.049138057976961136
0.0475226528942585
0.046007443219423294
0.0445835180580616
0.04324302449822426
0.041978977620601654
0.040785033255815506
0.039655651897192
0.03858580067753792
0.037570975720882416
0.036607030779123306
0.03569037839770317
0.034817591309547424
0.033985648304224014
0.033191803842782974
0.03243350237607956
0.0317084863781929
0.031014572829008102
0.030349910259246826
0.0297125969082117

In [0]:
# 테스트 크롤링

In [21]:
index2class = {v:k for k,v in class2index.items()}

for test in test_data:

  # 테스트 데이터 불러오기
    X = kor_nlp.morphs(test[0])
    X = Variable(make_BoW(X,word2index)).view(1,-1)
    
    pred = model(X)
    
    # 3가지 클래스 분류 중에 가장 높은것 추출
    pred = pred.max(1)[1].item()
    print("Input : %s" % test[0])
    print("Prediction : %s" % index2class[pred])
    print("Truth : %s" % test[1])

Input : 운전이 하고싶어
Prediction : 운전자
Truth : 운전자
Input : 건강식 챙겨먹고 있어
Prediction : 건강
Truth : 건강
Input : 뼈가 부러져서 금이 갔어
Prediction : 건강
Truth : 상해
Input : 사고나면 어떻게 해야돼?
Prediction : 운전자
Truth : 운전자
